In [10]:
library(tidyverse)
library(repr)
library(tidymodels)
library(janitor)
options(repr.matrix.max.rows = 6)


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [11]:
#Because this dataframe involves comma separated values, we used the appropriate read function.
apple <- read_csv("data/apple_quality.csv") |>
    head(-1) #This function removes the last row of the dataframe which included NA values and text crediting the author.
apple

Rows: 4001 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Acidity, Quality
dbl (7): A_id, Size, Weight, Sweetness, Crunchiness, Juiciness, Ripeness

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
0,-3.9700485,-2.512336,5.346330,-1.0120087,1.8449004,0.32983980,-0.491590483,good
1,-1.1952172,-2.839257,3.664059,1.5882323,0.8532858,0.86753008,-0.722809367,good
2,-0.2920239,-1.351282,-1.738429,-0.3426159,2.8386355,-0.03803333,2.621636473,bad
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3997,-2.6345153,-2.138247,-2.4404613,0.6572229,2.199709,4.7638592,-1.334611391,bad
3998,-4.0080037,-1.779337,2.3663970,-0.2003294,2.161435,0.2144884,-2.229719806,good
3999,0.2785397,-1.715505,0.1212173,-1.1540748,1.266677,-0.7765715,1.599796456,good


In [22]:
apple_manipulated <- apple |>
    clean_names() |>
    mutate(acidity = as.numeric(acidity))
apple_manipulated

a_id,size,weight,sweetness,crunchiness,juiciness,ripeness,acidity,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0,-3.9700485,-2.512336,5.346330,-1.0120087,1.8449004,0.32983980,-0.4915905,good
1,-1.1952172,-2.839257,3.664059,1.5882323,0.8532858,0.86753008,-0.7228094,good
2,-0.2920239,-1.351282,-1.738429,-0.3426159,2.8386355,-0.03803333,2.6216365,bad
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3997,-2.6345153,-2.138247,-2.4404613,0.6572229,2.199709,4.7638592,-1.334611,bad
3998,-4.0080037,-1.779337,2.3663970,-0.2003294,2.161435,0.2144884,-2.229720,good
3999,0.2785397,-1.715505,0.1212173,-1.1540748,1.266677,-0.7765715,1.599796,good


In [26]:
set.seed(4321)
#We selected the values we would use for our data analysis, which only included Ripeness, Acidity, and Quality.
#A random sample of 100 was taken because the data set was so large.
apple_select <- apple_manipulated |>
    select(ripeness, acidity, quality)
apple_select

ripeness,acidity,quality
<dbl>,<dbl>,<chr>
0.32983980,-0.4915905,good
0.86753008,-0.7228094,good
-0.03803333,2.6216365,bad
⋮,⋮,⋮
4.7638592,-1.334611,bad
0.2144884,-2.229720,good
-0.7765715,1.599796,good


In [8]:
# INSERT CROSS VALIDATION HERE (JUNYI)
apple_split <- initial_split(apple_select, prop = 0.75, strata = quality)
apple_train <- training(apple_split)
apple_test <- testing(apple_split)

apple_recipe <- recipe(quality ~ ripeness + acidity, data = apple_train) |>
 step_scale(all_predictors())|>
 step_center(all_predictors())

apple_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
         set_engine("kknn") |>
         set_mode("regression")

apple_vfold<- vfold_cv(apple_train, v=5, strata=quality)

apple_wkflw <- workflow() |>
    add_recipe(apple_recipe) |>
    add_model(apple_spec) 
apple_wkflw

gridvals<-tibble(neighbors=seq(1,200))

apple_results<- apple_wkflw |>
 tune_grid(resemples= apple_vfold, grid=gridvals) |>
 collect_metrics()
apple_results



ERROR: [1m[33mError[39m in `mc_cv()`:[22m
[33m![39m Can't subset columns that don't exist.
[31m✖[39m Column `quality` doesn't exist.


In [ ]:
apple_split <- initial_split(apple_select, prop = 0.75, strata = Quality)
apple_train <- training(apple_split)
apple_test <- testing(apple_split)

apple_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 



In [ ]:


# # apple_split <- initial_split(apple_select, prop = 0.75, strata = Quality)
# # apple_train <- training(apple_split)
# # apple_test <- testing(apple_split)

# # apple_recipe <- recipe(Quality ~ Ripeness + Acidity, data = apple_train) #Data is already scaled and centered, so there is no need to use step_scale.

# # knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
# #         set_engine("kknn") |>
# #         set_mode("classification")

# # knn_fit <- workflow() |>
# #     add_recipe(apple_recipe) |>
# #     add_model(knn_spec) |>
# #     fit(data = apple_train)

# # validation_predicted <- predict(knn_fit, apple_test) |>
# #     bind_cols(apple_test)

# # set.seed(1)

# # apple_vfold<- vfold_cv(apple_train, v=5, strata=Quality)

# # apple_recipe<- recipe(Quality~.,data=apple_train)|>
# #  step_scale(all_predictors())|>
# #  step_center(all_predictors())

# # knn_spec<-nearest_neighbor(weight_func="rectangular", neighbors=tune())|>
# # set_engine("kknn")|>
# # set_mode("classification")

# # knn_results<-workflow() |>
# #  add_recipe(apple_recipe) |>
# #  add_model(knn_spec) |>
# #  tune_grid(resamples=apple_vfold, grid=10) |>
# #  collect_metrics()
# # knn_results


# set.seed(1)
# apple_split<-initial_split(selected_data, prop=0.75, strata=Quality)
# apple_train<-training(apple_split)
# apple_test<-testing(apple_split)
# glimpse(apple_train)
# #create the split


In [ ]:

#In the following code, a K-nn classifier is created using the training data and cross validation with 10 folds is completed to select an optimal K value.

#quality_good_data <- data |>
#    select(Ripeness, Acidity, Quality) |>
#    filter(Quality == "good", na.rm = TRUE)

#quality_good_data

#options(repr.plot.width = 5, repr.plot.height = 5)
#plot_filtered_good_data <- quality_good_data  |>
#    ggplot(aes(x= Ripeness, y = Acidity, color = Quality)) +
#    geom_point(alpha =0.4) +
#    labs(x = "Ripeness",
#         y = "Acidity",
#         color = "Quality")
#plot_filtered_good_data

# options(repr.plot.width = 5, repr.plot.height = 5)

# quality_bad_data <- data |>
#     select(Ripeness, Acidity, Quality) |>
#     filter(Quality == "bad", na.rm = TRUE)

# quality_bad_data
# plot_filtered_bad_data <- quality_bad_data  |>
#     ggplot(aes(x= Ripeness, y = Acidity, color = Quality)) +
#     geom_point(alpha =0.4) +
#     labs(x = "Ripeness",
#          y = "Acidity",
#          color = "Quality")
# plot_filtered_bad_data